# LangChain

Large Language Models (LLM) have become increasingly used in more and more complex applications. When we use LLM in applications such as chatbots, there are many things that are happening that need to be considered:
- How do we keep track of historical context?
- Can we simulate a conversation with different people / agents?
- Can we direct the conversation to different 'people' depending on what is asked? (e.g. asking a panel of experts, and picking the most relevant expert based on the question)

We need a framework that allows us to abstract away many of these complexities, and ideally make it easy to logically structure how we use these LLMs. Introducing LangChain:

LangChain is an open-source framework for developing applications powered by LLM. This framework includes the ability to:
- Efficiently integrate with popular AI platforms such as OpenAI (company behind ChatGPT) and Hugging Face
- Connecting language driven models to data sources
- Enable LLMs to interact dynamically with their environment

It is designed to have modular components, that when combined together, can be used in many different applications.

## Course outline
- Models, Prompts and Output Parsers
    - Calling OpenAI
    - OpenAI Endpoints
    - Prompt templates
    - Using LangChain
- Handling memory
    - ConversationBufferMemory
    - How do LLM store memory?
    - ConversationBufferWindowMemory
    - ConversationTokenBufferMemory
    - ConverastionSummaryBufferMemory
    - Other memory methods
- Chains
    - What is a chain?
    - LLMChain
    - SimpleSequentialChain
    - SequentialChain
    - RouterChain
    - Other chains to explore

## Models, Prompts and Output Parsers

### Calling OpenAI

Before we look into LangChain and what it can do, we will make direct calls to OpenAI to show you what LLMs can do.

Let's look at an example to see how this works.

If want to follow along, you will need your own OpenAI API key.

Follow this link to get your own API key:
https://platform.openai.com/account/api-keys

In [13]:
# !pip install python-dotenv
# !pip install openai

In [29]:
# Put your API key here
os.environ["OPENAI_API_KEY"] = ""

In [30]:
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

### OpenAI Endpoints

An API (application programming interface) is a software intermediary that allows applications to talk to each other. An API endpoint is a specific location within an API that accepts requests and sends responses back.

OpenAI has several API endpoints that are offered. This includes tasks such as:
- Audio files to text
- Chat responses given list of messages
- Predicted text completion
- Create vector embeddings given a text input
- Generate images given prompts and image
- and many more...

It is fascinating how simple it is now to access all this through a simple API call.

We will demonstrate using openAI's ChatCompletion API. This is an API that is useful for having conversations, where given a list of messages comprising a conversation, it will return a response (think like a Chatbot). Let's start by keeping it simple, and just demonstrate what it looks like to call this API endpoint.

Let us choose ChatGPT as the LLM for this demo. We account for the deprecation of the LLM by comparing to the target date of June 12th 2024.

In [31]:
# account for deprecation of LLM model
import datetime

current_date = datetime.datetime.now().date() # Get the current date
target_date = datetime.date(2024, 6, 12) # Define the date after which the model should be set to "gpt-3.5-turbo"

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

We will now define a function that will take in a prompt, feed it to our chosen LLM, then return the response.

In [32]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

In [33]:
# In this converations, we have different roles (e.g. a customer, and an assistant), and also a 'system' role, which you can think of background information that defines who the assistant is.

In [34]:
get_completion(prompt="What is the capital of Australia?")

'The capital of Australia is Canberra.'

Here, we simply called the ChatCompletion API endpoint with a simple prompt, and got a response. A further breakdown of the function:
- messages : this is the list of messages we are going to pass to the ChatCompletion API endpoint
- response : this is the API endpoint, that takes the messages as input, uses the GPT-3.5 model to process and return a response
- temperature : this is a parameter that defines how random the response should be. 0 is telling the model to be more deterministic.

Let's now explore what else we can do with this.

### Prompt templates

Let's now customise the prompt/input, so that it is more dynamic, and like a template we can reuse.

Suppose we have a customer review of a restaurant they went to. It is written in a very rude tone. We want to be able to rewrite the review so that it is written more politely. We can design this as a prompt template which takes as inputs:
- customer review : content of the review
- style : what style to rewrite the review

We wrap this in a prompt text, which takes the 'customer review' and 'style' as dynamic inputs.


In [93]:
customer_review = """
The food in this restaurant is honestly the \
worst I have ever had. The steak was so dry, \
it was like the cow went to the desert and died again, \
and the portion was so small. The staff \
were not helpful, took forever to come \
and didn't seem to care about providing \
a good customer experience. The meal was also \
grossly overpriced. Do not come here if you \
want good food.
"""

In [94]:
style = """English in a polite tone."""

In [95]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_review}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is English in a polite tone..
text: ```
The food in this restaurant is honestly the worst I have ever had. The steak was so dry, it was like the cow went to the desert and died again, and the portion was so small. The staff were not helpful, took forever to come and didn't seem to care about providing a good customer experience. The meal was also grossly overpriced. Do not come here if you want good food.
```



In [96]:
response = get_completion(prompt)

In [97]:
response

'I must express my disappointment with the food served at this restaurant. Unfortunately, the steak I ordered was extremely dry and the portion size was quite small. Additionally, the staff were not very attentive and seemed disinterested in providing a positive customer experience. Furthermore, the cost of the meal was quite high in comparison to the quality of the food. I would not recommend this restaurant to anyone seeking a satisfying dining experience.'

We now have a more dynamic template, where we can feed in some content (the review) and a style, and the response will vary accordingly. In the next section, we'll repeat this exercise but using LangChain.

### Using LangChain

At the most basic level, we can think of there as being 3 components that make up a call to a LLM.
We need:
- A prompt (i.e. input) that will be fed into a LLM
- A large language model that will read in the prompt as input and process it
- A parser to take the output from the LLM and return it in a desired way

Let's repeat the same exercise we did with OpenAI, but using LangChain this time.

In [40]:
#!pip install --upgrade langchain

In [67]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# define prompt template
# input variables are denoted in {}
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string) # define prompt template
chat = ChatOpenAI(temperature=0.0, model=llm_model) # define API endpoint / LLM

# provide example prompt based off template design
writing_style = """English in a formal polite tone."""
customer_review = """
The food in this restaurant is honestly the \
worst I have ever had. The steak was so dry \
and the portion was so small. The staff \
were not helpful, took forever to come \
and didn't seem to care about providing \
a good customer experience. The meal was also \
grossly overpriced. Do not come here if you \
want good food.
"""
customer_messages = prompt_template.format_messages(
                    style=writing_style,
                    text=customer_review)

# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

# display response
print(customer_response.content)

I must express my disappointment with the quality of the food served at this establishment. Regrettably, the steak I ordered was excessively dry and the portion size was inadequate. Furthermore, the staff were unhelpful, took an unreasonable amount of time to attend to our needs, and appeared indifferent to providing a satisfactory customer experience. Additionally, the cost of the meal was exorbitant. I would advise against dining here if you are seeking a pleasurable culinary experience.


When using LangChain this time, we did not simply pass in a dynamic string, but we actually created a ChatPromptTemplate imported from the LangChain library. This gives us more flexibility, and makes for more modular and clean code. For example, we can actually see what the input variables to this template are as below:

In [66]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

The benefits of this will become more obvious once we start using more complex logic. Let's see another example now with an output parser.

### Parsing LLM output with LangChain

We are going to look at another example, this time using a strucuted output parser. Suppose we have a listing description, and we want to extract specific information from this listing description, and output into a Pythin dictionary. This involves the following steps:
- Design a prompt template which clearly takes a listing description, and specifices what information we want from it, and what format the output should be
- Choose a LLM to process this prompt
- Define an output parser, which will process the output from LLM and return in specific format

Here, we will use as an example, the StructuredOutputParser.from_response_schemas() which will return a json from the response.

In [69]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [73]:
bedroom_schema = ResponseSchema(
    name="bedroom"
    ,description="How many bedrooms does this property have? Answer as a single number if known. If unsure, Answer as Unknown.")
school_schema = ResponseSchema(
    name="school"
    ,description="What schools are around the property? If this information is not found, output Unknown.")
amenity_schema = ResponseSchema(
    name="amenity"
    ,description="Extract any amenties in the property, and output them as a comma separated Python list.")

response_schemas = [
    bedroom_schema 
    ,school_schema
    ,amenity_schema]

We create multiple ResponseSchema, which reflects how we want to extract information using LLM, with a key. We put all of this into a response_schemas list, which will then be fed into a StructuredOutputParser.from_response_schemas() from LangChain to create and output_parser. We can then later use this output_parser to parse the outputs from LLM into a json.

Additionally, the output_parser also helpfully has format instructions built in based on the provided response_schemas, which we can feed into the prompt later.

In [75]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [77]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bedroom": string  // How many bedrooms does this property have? Answer as a single number if known. If unsure, Answer as Unknown.
	"school": string  // What schools are around the property? If this information is not found, output Unknown.
	"amenity": string  // Extract any amenties in the property, and output them as a comma separated Python list.
}
```


We are now ready to define the prompt template. We see that there are two variables here:
- text : This will be the listing description
- format_instructions: This is simply how the output should be formatted - which we actually already have from the output parser above

Note that the prompt itself still clearly states what information we want out of the listing description, and should match the keys in the format_instructions.

In [84]:
# define prompt template
listing_info_format = """\
For the following text, extract the following information:

bedroom: How many bedrooms does this property have? Answer as a single number if known. If unsure, Answer as Unknown.
school: What schools are around the property? If this information is not found, output Unknown.
amenity: Extract any amenties in the property, and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=listing_info_format)

# give example input/prompt
listing_description = """
Eva Building - Near New & Luxury Apartment with 2 Large Balconies
Stylishly appointed this near-new three-bedroom apartment is perfectly located in the building of Eva Lane Cove. showcases a bright and versatile floor plan with spacious living and beautiful riverside views. Just footsteps to Hughes Park and a short stroll to city buses, cafes', shops and the bustling village also local schools.

Features including:
* Situated in sought-after location, enjoy parkside and riverside views
* Generous 2 bedrooms plus a multi-function room
* Large 2 balconies all with East aspects
* Elegance 2-layer blackout curtains in the living area and bedrooms
* Spacious interiors with a versatile open-plan living and dining area
* Island modern kitchen with 'Millie' appliances, gas cooking and dishwasher
* Three bedrooms all with built-in, the main bedroom with ensuite
* Sparkling bathroom with floor-to-ceiling tiles
* Ducting Air conditioning
* Video intercom and internal laundry.
* Secure one car space and storage

Outgoings:
Strata levy:$1208.60 pq
Council rate: $359.00 pq
Water: $158.45 pq approx.
"""

messages = prompt.format_messages(
    text=listing_description, 
    format_instructions=format_instructions # give an output parser format defined previously
)

# feed input into model
response = chat(messages)

# parse model output into desired dictionary format
output_dict = output_parser.parse(response.content)

In [87]:
output_dict

{'bedroom': 3,
 'school': 'Unknown',
 'amenity': ['Island modern kitchen',
  'Gas cooking',
  'Dishwasher',
  'Built-in wardrobes',
  'Ensuite',
  'Ducting Air conditioning',
  'Video intercom',
  'Internal laundry',
  'Secure one car space and storage']}

Our final output is now in a dictionary as desired!

To summarise, the key benefits from using LangChain so far is to have modularised code. It's clear which section is defining what we want to extract from the text, how we want the output formatted etc. If we didn't use LangChain, we would need to think really hard about how we structure our prompt to capture exactly what we want, a very daunting task!

We've only just touched on the benefits of LangChain, lets keep exploring how else it can be helpful!

## Handling memory

When we have a conversation with someone, our replies depend on what was talked about earlier. We keep a history of the context in our mind, which shape how we then respond. So far in our examples, there is no history or context provided when a response is generated. Here, we will look at an example where we have a conversation with AI, just like we would with a person.

### ConversationBufferMemory

We import the required libraries, and then initialise:
- LLM as ChatOpenAPI with GPT
- initialise memory object, ConversationBufferMemory
- initialise a ConversationChain object from langchain, with specified LLM and memory

In [106]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [107]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

Let's now start a conversation.

In [100]:
conversation.predict(input="Hi, my name is Jeffrey. My favourite sport is tennis.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Jeffrey. My favourite sport is tennis.
AI:

> Finished chain.


"Hello Jeffrey, it's nice to meet you! Tennis is a great sport. Did you know that it originated in 12th century France as a game played with the palm of the hand? It wasn't until the 16th century that rackets were introduced. Today, tennis is played all over the world and is a popular spectator sport as well. Do you have a favourite tennis player?"

In [101]:
conversation.predict(input="My favourite player is Roger Federer.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Jeffrey. My favourite sport is tennis.
AI: Hello Jeffrey, it's nice to meet you! Tennis is a great sport. Did you know that it originated in 12th century France as a game played with the palm of the hand? It wasn't until the 16th century that rackets were introduced. Today, tennis is played all over the world and is a popular spectator sport as well. Do you have a favourite tennis player?
Human: My favourite player is Roger Federer.
AI:

> Finished chain.


"Ah, Roger Federer is a great choice! He has won a record 20 Grand Slam singles titles and has been ranked world No. 1 in men's singles tennis by the Association of Tennis Professionals a record total of 310 weeks. He is known for his elegant playing style and his ability to play well on all surfaces. Have you ever seen him play in person?"

In [102]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Jeffrey. My favourite sport is tennis.
AI: Hello Jeffrey, it's nice to meet you! Tennis is a great sport. Did you know that it originated in 12th century France as a game played with the palm of the hand? It wasn't until the 16th century that rackets were introduced. Today, tennis is played all over the world and is a popular spectator sport as well. Do you have a favourite tennis player?
Human: My favourite player is Roger Federer.
AI: Ah, Roger Federer is a great choice! He has won a record 20 Grand Slam singles titles and has been ranked world No. 1 in men's singles tennis by the Association of Tennis Professionals a record total of 310 weeks. 

'Your name is Jeffrey, as you mentioned earlier.'

We see that we can have a conversation with AI, then after multiple inputs, ask it again what is my name. The AI remembers my name even though it was from a conversation from awhile back. Using this conversation chain with the defined memory object, we can now store the historical conversation.

We can also print out the conversation up till now.

In [103]:
# see the stored history
print(memory.buffer)

Human: Hi, my name is Jeffrey. My favourite sport is tennis.
AI: Hello Jeffrey, it's nice to meet you! Tennis is a great sport. Did you know that it originated in 12th century France as a game played with the palm of the hand? It wasn't until the 16th century that rackets were introduced. Today, tennis is played all over the world and is a popular spectator sport as well. Do you have a favourite tennis player?
Human: My favourite player is Roger Federer.
AI: Ah, Roger Federer is a great choice! He has won a record 20 Grand Slam singles titles and has been ranked world No. 1 in men's singles tennis by the Association of Tennis Professionals a record total of 310 weeks. He is known for his elegant playing style and his ability to play well on all surfaces. Have you ever seen him play in person?
Human: What is my name?
AI: Your name is Jeffrey, as you mentioned earlier.


We can also explicitly set a conversation as context.

In [105]:
memory = ConversationBufferMemory()
memory.save_context(
    {"input": "Hi"}, 
    {"output": "What's up"})
print(memory.buffer)

Human: Hi
AI: What's up


### How do LLM store memory?

Large Language Models are stateless, meaning that each call to the API is independent. The memory that we see is actually from us providing the full conversation up to that point as the context. We saw earlier that the memory buffer had the entire conversation history stored. However, overtime as the conversation gets longer, the memory will eventually be too big and costly to keep passing in as input. LangChain provides many different ways to handle its memory. We have so far seen the most basic, ConversationBufferMemory() which just stores all the conversation up to that point as context. Let's explore other memories in LangChain.

### ConversationBufferWindowMemory

One way to manage storage of history is to only keep the last 'k' conversations in history. We can do this with the ConversationBufferWindowMemory, which has a parameter 'k'.

For example, below we set k=1. We can see from the loaded memory variables that it has only stored the most recent conversation.

In [109]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1) 
memory.save_context({"input": "Hi, my name is Jeffrey and I like tennis"},
                    {"output": "Hi Jeffrey, have you heard of Roger Federer?"})
memory.save_context({"input": "Yes, he is one of my favourite players."},
                    {"output": "He has over 20 grand slams."})

memory.load_memory_variables({})

{'history': 'Human: Yes, he is one of my favourite players.\nAI: He has over 20 grand slams.'}

We can demonstrate this in a conversation as well, where we can see below that the AI can not remember what my name is, even though it was introduced in the beginning. This is because the window is only k=1, so it only stores the most recent conversation.

In [110]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [111]:
conversation.predict(input="Hi, my name is Jeffrey.")

"Hello Jeffrey, it's nice to meet you. My name is OpenAI. How can I assist you today?"

In [114]:
conversation.predict(input="What is 1 + 1.")

'1 + 1 equals 2, as it is a basic arithmetic operation.'

In [115]:
conversation.predict(input="What is my name.")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

### ConversationTokenBufferMemory

Another way to store memory is to store a certain number of tokens (e.g. characters) in the history, rather than number of conversations. Conversations can vary in length, and some can be very long, so storing by tokens is a more consistent way of storing history from a memory perspective. Costs for API calls are usually also dependent on token length, so this method is more directly related to the cost.

In [126]:
# !pip install tiktoken

Let's start by setting the max token limit to 100. In this example, we see that it stores only the most recent conversation. This is because it is under 100 tokens, however if we were to include the conversation earlier it would be over 100.

In [124]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hi, my name is Jeffrey. My favourite sport is tennis."},
                    {"output": "Hello Jeffrey, it's nice to meet you! Tennis is a great sport. Did you know that it originated in 12th century France as a game played with the palm of the hand? It wasn't until the 16th century that rackets were introduced. Today, tennis is played all over the world and is a popular spectator sport as well. Do you have a favourite tennis player?"})
memory.save_context({"input": "My favourite player is Roger Federer."},
                    {"output": "Ah, Roger Federer is a great choice! He has won a record 20 Grand Slam singles titles and has been ranked world No. 1 in men's singles tennis by the Association of Tennis Professionals a record total of 310 weeks. He is known for his elegant playing style and his ability to play well on all surfaces. Have you ever seen him play in person?"})
memory.load_memory_variables({})

{'history': "Human: My favourite player is Roger Federer.\nAI: Ah, Roger Federer is a great choice! He has won a record 20 Grand Slam singles titles and has been ranked world No. 1 in men's singles tennis by the Association of Tennis Professionals a record total of 310 weeks. He is known for his elegant playing style and his ability to play well on all surfaces. Have you ever seen him play in person?"}

If instead we set the max token limit to 10, we see that it does not store anything, becase the most recent conversation has over 10 tokens. It will only store a conversation if it is under the max token limit.

In [125]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "Hi, my name is Jeffrey. My favourite sport is tennis."},
                    {"output": "Hello Jeffrey, it's nice to meet you! Tennis is a great sport. Did you know that it originated in 12th century France as a game played with the palm of the hand? It wasn't until the 16th century that rackets were introduced. Today, tennis is played all over the world and is a popular spectator sport as well. Do you have a favourite tennis player?"})
memory.save_context({"input": "My favourite player is Roger Federer."},
                    {"output": "Ah, Roger Federer is a great choice! He has won a record 20 Grand Slam singles titles and has been ranked world No. 1 in men's singles tennis by the Association of Tennis Professionals a record total of 310 weeks. He is known for his elegant playing style and his ability to play well on all surfaces. Have you ever seen him play in person?"})
memory.load_memory_variables({})

{'history': ''}

### ConversationSummaryBufferMemory

When conversations get really long, both the window and token methods are not satisfactory, since they will start dropping more and more of the conversations if costs are to be maintained. A neat trick to get around this is to use the ConversationSummaryBufferMemory. This object similarly has a max_token_limit like before, however when the memory is past this max_token_limit, it will summarise all the conversations up to now using LLM, and then keep the summarised conversations as the context instead. It will then process new conversations as usual until the max_token_limit is reach once again, and it will summarise the conversations thus far again, and so on.

Let's see an example.

In [131]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
listing_description = """
Eva Building - Near New & Luxury Apartment with 2 Large Balconies
Stylishly appointed this near-new three-bedroom apartment is perfectly located in the building of Eva Lane Cove. showcases a bright and versatile floor plan with spacious living and beautiful riverside views. Just footsteps to Hughes Park and a short stroll to city buses, cafes', shops and the bustling village also local schools.

Features including:
* Situated in sought-after location, enjoy parkside and riverside views
* Generous 2 bedrooms plus a multi-function room
* Large 2 balconies all with East aspects
* Elegance 2-layer blackout curtains in the living area and bedrooms
* Spacious interiors with a versatile open-plan living and dining area
* Island modern kitchen with 'Millie' appliances, gas cooking and dishwasher
* Three bedrooms all with built-in, the main bedroom with ensuite
* Sparkling bathroom with floor-to-ceiling tiles
* Ducting Air conditioning
* Video intercom and internal laundry.
* Secure one car space and storage

Outgoings:
Strata levy:$1208.60 pq
Council rate: $359.00 pq
Water: $158.45 pq approx.
"""

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=200)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "Give me a property listing,"}, 
                    {"output": f"{listing_description}"})

In [132]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI and asks for a property listing. The AI provides a detailed description of a luxury apartment with riverside views, spacious living areas, and modern amenities. The apartment is located near parks, cafes, shops, and schools. The outgoings include strata levy, council rate, and water fees.'}

In above example, we give the memory a long property listing. Since it is more than 200 tokens, it is summarised in a few short sentences and this then becomes its context/memory.

If we then prompt this further, the AI will reply, having the summarised context. The new AI response is now also part of the memory - with the latest response not being summarised since in total it is still under 200 tokens.

In [133]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)
conversation.predict(input="Do you think it is a good property?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks for a property listing. The AI provides a detailed description of a luxury apartment with riverside views, spacious living areas, and modern amenities. The apartment is located near parks, cafes, shops, and schools. The outgoings include strata levy, council rate, and water fees.
Human: Do you think it is a good property?
AI:

> Finished chain.


'Based on the features and location of the property, it appears to be a high-quality and desirable option for those seeking luxury living with convenient access to local amenities. However, whether or not it is a good property ultimately depends on individual preferences and needs.'

In [134]:
memory.load_memory_variables({})

{'history': 'System: The human greets the AI and asks for a property listing. The AI provides a detailed description of a luxury apartment with riverside views, spacious living areas, and modern amenities. The apartment is located near parks, cafes, shops, and schools. The outgoings include strata levy, council rate, and water fees.\nHuman: Do you think it is a good property?\nAI: Based on the features and location of the property, it appears to be a high-quality and desirable option for those seeking luxury living with convenient access to local amenities. However, whether or not it is a good property ultimately depends on individual preferences and needs.'}

### Other memory methods

There are many other memory types, and you are encouraged to have a look at what other options are there. These include:
- Vector data memory - stores text in a vector database and retrives the most relevant blocks of text
- Entity memories - remembers details about specific entities
- and more...

We can use multiple memories at one time, so go experiment!

## Chains

### What is a chain?

### LLMChain

In [137]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.9, model=llm_model)

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a product that {product}?"
)

chain = LLMChain(llm=llm, prompt=prompt)

product = "compares property listings"
chain.run(product)

'"Property Finder" or "Listing Compare" could be good names for a product that compares property listings.'

### SimpleSequentialChain

In [143]:
from langchain.chains import SimpleSequentialChain

In [144]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a product that {product_function}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [145]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    product:{product_description}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [146]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True
)

In [147]:
product_function = "compares property listings"
overall_simple_chain.run(product_function)



> Entering new SimpleSequentialChain chain...
"Listings Comparison Tool" or "Property Comparison Engine" are some good names to describe a product that compares property listings.
A tool that enables property buyers to easily compare different listings, making the search process quicker and more efficient.

> Finished chain.


'A tool that enables property buyers to easily compare different listings, making the search process quicker and more efficient.'

### SequentialChain

In [150]:
from langchain.chains import SequentialChain

In [154]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a product that {product_function}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="product_name")

In [155]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    product:{product_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="product_description")

In [157]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["product_function"],
    output_variables=["product_name","product_description"],
    verbose=True
)

In [159]:
product_function = "compares property listings"
overall_chain(product_function)



> Entering new SequentialChain chain...

> Finished chain.


{'product_function': 'compares property listings',
 'product_name': '"Listings Comparator" or "Property Search Comparator"',
 'product_description': 'Our Listings/Property Search Comparator is a tool that helps you compare and find the best real estate listings based on your preferences.'}

### RouterChain

### Other chains to explore